# Projeto: Análise de Anomalias em Série Temporal

BIMASTER - Trabalho de final de curso

Nome: Alex Marques Campos

Etapa 02.B: Detecção de anomalias via predição de séries

O objetivo deste notebook é carregar os dados das séries históricas de interesse e observar se é possível utilizar predição de séries temporais para realizar a detecção de anomalias. Para isso, um modelo LSTM.

O primeiro passo é carregar as bibliotecas necessárias ao processamento e os dados propriamente ditos, que estão armazenados nos arquivos CSV (_comma separated values_) armazenados no subdiretório './__dados__/'.

## Configuração do ambiente de execução

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from pathlib import Path

In [ ]:
NOME_DIRETORIO_DADOS = 'dados'
NOME_ARQUIVO_SERIE_IBCBR = 'serie_ibcbr.csv'
NOME_ARQUIVO_SERIE_IBCBR_RESIDUO = 'serie_ibcbr_residuo.csv'
NOME_ARQUIVO_SERIE_IBCBR_DIFERENCAS = 'serie_ibcbr_diferencas.csv'
NOME_ARQUIVO_SERIE_IBCBR_DIFERENCAS_RESIDUO = 'serie_ibcbr_diferencas_residuo.csv'

In [ ]:
print(f'pandas == {pd.__version__}')
print(f'matplotlib == {matplotlib.__version__}')

In [ ]:
# ajustamos o formato de apresentação padrão dos gráficos
sns.set_theme(style="white", palette="pastel")

In [ ]:
# habilitamos a visualização especial de dataframes disponível no
# Google Colaboratory, para facilitar a exploração dos dados.
from google.colab import data_table
data_table.enable_dataframe_formatter()

## Carga dos dados

Lembre-se de criar uma pasta chamada '/content/dados' no Google Colaboratory e enviar os arquivos '.csv' para essa pasta.

In [ ]:
path_dir_dados = Path('.') / NOME_DIRETORIO_DADOS

path_arq_ibcbr     = path_dir_dados / NOME_ARQUIVO_SERIE_IBCBR
path_arq_ibcbr_res = path_dir_dados / NOME_ARQUIVO_SERIE_IBCBR_RESIDUO
path_arq_dif       = path_dir_dados / NOME_ARQUIVO_SERIE_IBCBR_DIFERENCAS
path_arq_dif_res   = path_dir_dados / NOME_ARQUIVO_SERIE_IBCBR_DIFERENCAS_RESIDUO

In [ ]:
def carregar_csv(path_csv:Path) -> pd.DataFrame:
  """
  Carrega os dados de um arquivo CSV em um dataframe de
  forma padronizada no escopo do projeto.
  path_csv : objeto Path que aponta para o arquivo.
  """
  if (path_csv is None) or (not path_csv.is_file()):
    raise ValueError("The given path object doesn't point to a valid csv file.")

  return pd.read_csv(path_csv,
                   sep=',',
                   parse_dates=True,
                   infer_datetime_format=True,
                   index_col=0,
                   decimal='.',
                   encoding='utf8')

In [ ]:
# carregamos os arquivos de dados em dataframes, para iniciar a análise
df_ibcbr     = carregar_csv(path_arq_ibcbr)
df_ibcbr_res = carregar_csv(path_arq_ibcbr_res)
df_dif       = carregar_csv(path_arq_dif)
df_dif_res   = carregar_csv(path_arq_dif)

In [ ]:
def verificar_propriedades(id:int, nome:str, dataframe:pd.DataFrame) -> None:
  """
  Imprime propriedades do dataframe nomeado para inspeção visual dos dados.
  id: inteiro que identifica o dataframe.
  nome: nome do dataframe
  dataframe: objeto do dataframe
  """
  print(f'[{id:03d}] Dataframe: {nome}')
  print('-' * 5)
  print(f'Shape: {dataframe.shape}')
  print('-' * 5)
  print(dataframe.head(4))
  print('')

In [ ]:
# verificamos se os dataframes foram carregados corretamente.
series = {
  'IBC-BR': df_ibcbr,
  'IBC-BR (resíduo)': df_ibcbr_res,
  'Diferenças': df_dif,
  'Diferenças (resíduo)': df_dif_res
}

for idx,serie in enumerate(series.items()):
  verificar_propriedades(idx+1, serie[0], serie[1])

Neste ponto, temos todos os dados carregados em DataFrames pandas e prontos para análise.

## Predição de Série Temporal

### LSTM

Nossa primeira abordagem será treinar uma rede neural LSTM para fazer predição da série temporal IBC-BR original. Primeiro, treinaremos uma rede LSTM com todos os dados da série, para identificar os melhores hiper-parâmetros do modelo. Depois, dividiremos os dados em blocos de 12 meses e treinaremos com um número N de blocos, variando entre N = 1 e N = QUANTIDADE_DADOS mod 12) - 1, realizando predições para os próximos 12 meses e comparando as predições com os dados obtidos.

Começamos definindo funções auxiliares ao processo de análise.

In [ ]:
def configurar_ticks_anuais(ax):
  '''
  Procedimento auxiliar para configurar a exibição do eixo x
  de um gráfico do matplotlib para dados de séries temporais
  para os quais só sejam marcados os valores ano a ano.
  '''
  ax.xaxis.set_major_locator(matplotlib.dates.YearLocator(base=1))
  ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%Y"))

In [ ]:
def desenhar_serie_com_anomalias(df:pd.DataFrame,
                                 indice_anomalias:pd.core.indexes.datetimes.DatetimeIndex,
                                 nome_serie:str='IBC-BR') -> None:
  fig = plt.figure(figsize=(16,5))

  plt.plot(df['valor'], 'b', label=f'Índice {nome_serie}')

  for e in indice_anomalias:
    plt.axvline(x=e, color='r', linestyle=':')

  plt.legend(loc='best')
  plt.ylabel(nome_serie)
  plt.xlabel('Tempo')

  # ajustamos o eixo x da figura para exibir ticks a cada ano
  configurar_ticks_anuais(fig.axes[0])

  plt.show()